# Processing eROSITA data

This tutorial will teach you how to use DAXA to process raw eROSITA data into a science ready state using one line of Python code (or several lines, if you wish to have more control over the settings for each step). **This relies on there being an initialised (either manually before launching Python, or in your bash profile/rc) backend installation of the eROSITA Science Analysis Software System (eSASS), including accessible calibration files** - DAXA will check for such an installation, and will not allow processing to start without it.

**All DAXA processing steps will parallelise as much as possible - processes running on different ObsIDs/instruments/sub-exposures will be run simultaneously (if cores are available)**

## Import Statements

In [1]:
from daxa.mission import eRASS1DE
from daxa.archive import Archive
from daxa.process.simple import full_process_erosita
from daxa.process.erosita.clean import flaregti
from daxa.process.erosita.assemble import cleaned_evt_lists

## An Archive to be processed

Every processing function implemented in DAXA takes an Archive instance as its first argument; if you don't already know what that is then you should go back and read the following tutorials:

* [Creating a DAXA archive](archives.html) - This explains how to create an archive, load an existing archive, and the various properties and features of DAXA archives.
* [Using DAXA missions](missions.html) - Here we explain what DAXA mission classes are and how to use them to select only the data you need.

Here we create an archive of eRASS DR1 observations of the galaxy cluster Abell 3266:

In [2]:
# er = eRASS1DE()
# er.filter_on_name('A3266')
# arch = Archive('Abell3266', er, clobber=True)

from daxa.mission import eROSITACalPV

ec = eROSITACalPV()
ec.filter_on_positions([[133.071, -1.025]])
arch = Archive('eFEDSXCS-339', ec, clobber=True)

/mnt/pact/dt237/code/PycharmProjects/DAXA/daxa/mission/base.py:1075: UserWarning: A field-of-view cannot be easily defined for eROSITACalPV and this number is the approximate half-length of an eFEDS section, the worst case separation - this is unnecessarily large for pointed observations, and you should make your own judgement on a search distance.
  fov = self.fov


## One-line solution

Though we provide individual functions that wrap the various steps required to reduce and prepare eROSITA data, and they can be used separately for greater control over the configuration parameters, we also include a one-line solution which executes the processing steps with default configuration.

We believe that the default parameters are adequate for most use cases, and this allows for users unfamiliar with the intricacies of eROSITA data to easily start working with it. Executing the following will automatically generate cleaned combined event lists for all telescope modules selected upon the original declaration of the mission.

fm00_300007_020_EventList_c001.fits

In [3]:
full_process_erosita(arch)

eROSITACalPV - Generating final event lists: 100%|██████████████████████████████| 2/2 [00:58<00:00, 29.24s/it]


## General arguments for all processing functions

There are some arguments that all processing functions will take - they don't control the behaviour of the processing step itself, but instead how the commands are executed:

* **num_cores** - The number of cores that the function is allowed to use. This is set globally by the daxa.NUM_CORES constant (if set before any other DAXA function is imported), and by default is 90% of the cores available on the system.
* **timeout** - This controls how long an individual instance of the process is allowed to run before it is cancelled; the whole function may run for longer depending how many pieces of data need processing and how many cores are allocated. The default is generally null, but it can be set using an astropy quantity with time units.

## Breaking down the eROSITA processing steps

There are fewer individual steps for eROSITA compared to a telescope like XMM - this reflects its simpler design, with only a single camera type, as well as the differences in how data are served to the community and the backend software design. This section deals describes the different processing steps that DAXA can apply to eROSITA data (both all-sky and calibration).

### Identifying periods of high flaring (flaregti)

[flaregti function](../../daxa.process.erosita.html#daxa.process.erosita.clean.flaregti)

### Applying event filtering and good-time-intervals to the raw event lists (cleaned_evt_lists)

[cleaned_evt_lists function](../../daxa.process.erosita.html#daxa.process.erosita.assemble.cleaned_evt_lists)